##Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.cluster import KMeans
import plotly.graph_objects as go

##Loading the dataset

In [6]:
df = pd.read_csv('/content/marketing_campaign.csv',delimiter = '\t')

##Formatting the data

In [7]:
current_date = date(2021,11,3)
df['member_since']=pd.to_datetime(df['Dt_Customer'], dayfirst=True,format = '%d-%m-%Y')
df['member_since'] = (pd.to_numeric(df['member_since'].dt.date.apply(lambda x: (current_date - x)).dt.days, downcast='integer')/30).astype(int)

In [8]:
df['Age']=2021-df['Year_Birth']
df['Spending']=df['MntWines']+df['MntFruits']+df['MntMeatProducts']+df['MntFishProducts']+df['MntSweetProducts']+df['MntGoldProds']

In [9]:
df['Marital_Status']=df['Marital_Status'].replace({'Divorced':1,'Single':1,'Married':2,'Together':2,'Absurd':1,'Widow':1,'YOLO':1, 'Alone':1})

In [10]:
df['TotalFamilyMembers'] = df['Marital_Status'].astype(int) + df['Kidhome'].astype(int) + df['Teenhome'].astype(int)

In [11]:
df.dropna(inplace=True)
df=df[df['Income']<600000]
df.reset_index(inplace=True, drop = True)

##Normalising the data

In [12]:
scaler=StandardScaler()
dataset_temp=df[['Income', 'member_since', 'NumWebPurchases']]
X_std=scaler.fit_transform(dataset_temp)
X = normalize(X_std,norm='l2')

##Clustering

In [13]:
kmeans = KMeans(n_clusters=4,max_iter=10000, n_init=100).fit(X)
kmeans.labels_

array([0, 2, 3, ..., 2, 3, 1], dtype=int32)

In [14]:
df['labels'] = kmeans.labels_

##Visualising the clustering

In [15]:
PLOT = go.Figure()
for C in list(df.labels.unique()):
    

    PLOT.add_trace(go.Scatter3d(x = df[df.labels == C]['Income'],
                                y = df[df.labels == C]['member_since'],
                                z = df[df.labels == C]['NumWebPurchases'],                        
                                mode = 'markers',marker_size = 6, marker_line_width = 1,
                                name = str(C)))
PLOT.update_traces(hovertemplate='Income: %{x} <br>member_since: %{y} <br>NumWebPurchases: %{z}')

    
PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                   scene = dict(xaxis=dict(title = 'Income', titlefont_color = 'black'),
                                yaxis=dict(title = 'member_since', titlefont_color = 'black'),
                                zaxis=dict(title = 'NumWebPurchases', titlefont_color = 'black')),
                   font = dict(family = "Gilroy", color  = 'black', size = 12))